In [ ]:
# Chattea Chatbot - RAG Approach with Sentence Transformers
# Retrieval-Augmented Generation for Natural Customer Support

"""
INSTALLATION REQUIRED:
!pip install sentence-transformers numpy

Note: sentence-transformers is open-source (Apache 2.0) by HuggingFace
"""

In [1]:
import json
import numpy as np
from sentence_transformers import SentenceTransformer
import re

# ============================================================================
# SECTION 1: PREPROCESSING & UTILITIES
# ============================================================================

def preprocess_text(text):
    """
    Clean text for better embedding quality
    Args:
        text (str): Raw input text
    Returns:
        str: Cleaned text
    """
    # Convert to lowercase
    text = text.lower()
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

c:\Users\nicho\Documents\GitHub_Repository\Python\chattea-ai\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ============================================================================
# SECTION 2: RAG VECTOR STORE
# ============================================================================

class VectorStore:
    """
    Simple vector store for semantic search
    Stores embeddings and retrieves most similar items
    """
    
    def __init__(self, model_name='paraphrase-multilingual-MiniLM-L12-v2'):
        """
        Initialize vector store with embedding model
        Args:
            model_name (str): Sentence transformer model name
        """
        print(f"Loading embedding model: {model_name}")
        print("This may take a moment on first run...")
        
        # Load model (supports Indonesian + English!)
        self.model = SentenceTransformer(model_name)
        
        # Storage
        self.embeddings = []
        self.documents = []
        self.metadata = []
        
        print(f"✓ Model loaded! Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
    
    def add_documents(self, documents, metadata=None):
        """
        Add documents to vector store
        Args:
            documents (list): List of text strings
            metadata (list): Optional metadata for each document
        """
        if metadata is None:
            metadata = [{}] * len(documents)
        
        print(f"Encoding {len(documents)} documents...")
        
        # Clean documents
        cleaned_docs = [preprocess_text(doc) for doc in documents]
        
        # Generate embeddings
        embeddings = self.model.encode(cleaned_docs, show_progress_bar=True)
        
        # Store
        self.embeddings.extend(embeddings)
        self.documents.extend(documents)
        self.metadata.extend(metadata)
        
        print(f"✓ Added {len(documents)} documents to vector store")
        print(f"✓ Total documents: {len(self.documents)}")
    
    def search(self, query, top_k=3):
        """
        Search for most similar documents
        Args:
            query (str): Search query
            top_k (int): Number of results to return
        Returns:
            list: List of tuples (document, similarity_score, metadata)
        """
        if not self.embeddings:
            return []
        
        # Encode query
        query_cleaned = preprocess_text(query)
        query_embedding = self.model.encode([query_cleaned])[0]
        
        # Calculate cosine similarity with all documents
        similarities = self._cosine_similarity(query_embedding, np.array(self.embeddings))
        
        # Get top K indices
        top_indices = np.argsort(similarities)[::-1][:top_k]
        
        # Prepare results
        results = []
        for idx in top_indices:
            results.append({
                'document': self.documents[idx],
                'similarity': float(similarities[idx]),
                'metadata': self.metadata[idx]
            })
        
        return results
    
    def _cosine_similarity(self, vec1, vec2):
        """
        Calculate cosine similarity between vector and matrix
        Args:
            vec1 (np.array): Single vector
            vec2 (np.array): Matrix of vectors
        Returns:
            np.array: Similarity scores
        """
        # Normalize vectors
        vec1_norm = vec1 / (np.linalg.norm(vec1) + 1e-10)
        vec2_norm = vec2 / (np.linalg.norm(vec2, axis=1, keepdims=True) + 1e-10)
        
        # Dot product
        return np.dot(vec2_norm, vec1_norm)


In [3]:

# ============================================================================
# SECTION 3: RAG CHATBOT
# ============================================================================

class ChatteraRAGChatbot:
    """
    RAG-based chatbot for Chattea customer support
    Uses semantic search to find relevant responses
    """
    
    def __init__(self, intents_data, responses_data, 
                 model_name='paraphrase-multilingual-MiniLM-L12-v2',
                 similarity_threshold=0.3):
        """
        Initialize RAG chatbot
        Args:
            intents_data (list): List of dicts with 'intent' and 'text' examples
            responses_data (dict): Intent -> response mapping
            model_name (str): Sentence transformer model
            similarity_threshold (float): Minimum similarity to accept match
        """
        self.intents_data = intents_data
        self.responses_data = responses_data
        self.similarity_threshold = similarity_threshold
        
        # Initialize vector store
        self.vector_store = VectorStore(model_name=model_name)
        
        # Build knowledge base
        self._build_knowledge_base()
    
    def _build_knowledge_base(self):
        """
        Build vector database from training data
        """
        print("\n" + "=" * 60)
        print("BUILDING KNOWLEDGE BASE")
        print("=" * 60)
        
        documents = []
        metadata = []
        
        # Add all intent examples to vector store
        for item in self.intents_data:
            documents.append(item['text'])
            metadata.append({'intent': item['intent']})
        
        # Add responses as additional context (optional but helpful)
        for intent, response in self.responses_data.items():
            documents.append(response)
            metadata.append({'intent': intent, 'is_response': True})
        
        self.vector_store.add_documents(documents, metadata)
        print("=" * 60 + "\n")
    
    def chat(self, user_input, top_k=3, verbose=True):
        """
        Get chatbot response using RAG
        Args:
            user_input (str): User's message
            top_k (int): Number of similar documents to retrieve
            verbose (bool): Print debug info
        Returns:
            str: Bot's response
        """
        # Retrieve similar documents
        results = self.vector_store.search(user_input, top_k=top_k)
        
        if not results:
            return "Maaf, saya tidak mengerti pertanyaan Anda."
        
        # Get best match
        best_match = results[0]
        
        if verbose:
            print(f"\n{'=' * 60}")
            print(f"🔍 RETRIEVAL RESULTS FOR: '{user_input}'")
            print(f"{'=' * 60}")
            for i, result in enumerate(results, 1):
                intent = result['metadata'].get('intent', 'unknown')
                similarity = result['similarity']
                doc_preview = result['document'][:80] + "..." if len(result['document']) > 80 else result['document']
                print(f"\n{i}. Intent: {intent}")
                print(f"   Similarity: {similarity:.4f}")
                print(f"   Document: {doc_preview}")
            print(f"{'=' * 60}\n")
        
        # Check if similarity is high enough
        if best_match['similarity'] < self.similarity_threshold:
            return "Maaf, saya tidak yakin apa yang Anda maksud. Bisa tolong diperjelas?"
        
        # Get intent and return response
        intent = best_match['metadata'].get('intent')
        response = self.responses_data.get(intent, "Maaf, saya tidak mengerti.")
        
        if verbose:
            print(f"✓ Selected Intent: {intent}")
            print(f"✓ Confidence: {best_match['similarity']:.4f}\n")
        
        return response
    
    def batch_test(self, test_cases):
        """
        Test chatbot on multiple inputs
        Args:
            test_cases (list): List of test strings
        """
        print("\n" + "=" * 60)
        print("BATCH TESTING")
        print("=" * 60)
        
        for i, test_input in enumerate(test_cases, 1):
            print(f"\n--- Test Case {i} ---")
            print(f"👤 User: {test_input}")
            response = self.chat(test_input, verbose=False, top_k=1)
            print(f"🤖 Bot: {response}")
        
        print("\n" + "=" * 60)

In [4]:

# ============================================================================
# SECTION 4: DATASET & DEMO
# ============================================================================

# Expanded dataset for Chattea (Indonesian + English support)
intents_dataset = [
    # Blast message intents (more variations!)
    {"intent": "blast_message", "text": "cara blast message"},
    {"intent": "blast_message", "text": "kirim pesan massal"},
    {"intent": "blast_message", "text": "bagaimana mengirim blast"},
    {"intent": "blast_message", "text": "blast pesan ke banyak orang"},
    {"intent": "blast_message", "text": "how to send bulk messages"},
    {"intent": "blast_message", "text": "broadcast pesan whatsapp"},
    {"intent": "blast_message", "text": "kirim ke semua kontak"},
    {"intent": "blast_message", "text": "mass messaging feature"},
    
    # Schedule message intents
    {"intent": "schedule_message", "text": "jadwalkan pesan"},
    {"intent": "schedule_message", "text": "cara schedule message"},
    {"intent": "schedule_message", "text": "kirim pesan terjadwal"},
    {"intent": "schedule_message", "text": "atur waktu kirim pesan"},
    {"intent": "schedule_message", "text": "schedule WhatsApp message"},
    {"intent": "schedule_message", "text": "pesan otomatis terkirim"},
    {"intent": "schedule_message", "text": "delayed message sending"},
    
    # Filter WhatsApp number
    {"intent": "filter_number", "text": "filter nomor whatsapp"},
    {"intent": "filter_number", "text": "cek nomor terdaftar"},
    {"intent": "filter_number", "text": "validasi nomor whatsapp"},
    {"intent": "filter_number", "text": "nomor aktif whatsapp"},
    {"intent": "filter_number", "text": "check valid WhatsApp numbers"},
    {"intent": "filter_number", "text": "verifikasi nomor wa"},
    {"intent": "filter_number", "text": "which numbers are registered"},
    
    # Panasin WhatsApp
    {"intent": "panasin_wa", "text": "panasin whatsapp"},
    {"intent": "panasin_wa", "text": "cara panasin wa"},
    {"intent": "panasin_wa", "text": "hindari blokir whatsapp"},
    {"intent": "panasin_wa", "text": "warming up whatsapp"},
    {"intent": "panasin_wa", "text": "prevent WhatsApp ban"},
    {"intent": "panasin_wa", "text": "avoid getting blocked"},
    {"intent": "panasin_wa", "text": "warm up account"},
    
    # General help
    {"intent": "help", "text": "bantuan"},
    {"intent": "help", "text": "tolong saya"},
    {"intent": "help", "text": "help"},
    {"intent": "help", "text": "apa yang bisa dilakukan"},
    {"intent": "help", "text": "what can you do"},
    {"intent": "help", "text": "fitur apa saja"},
    {"intent": "help", "text": "features available"},
    
    # Greeting
    {"intent": "greeting", "text": "halo"},
    {"intent": "greeting", "text": "hi"},
    {"intent": "greeting", "text": "hai"},
    {"intent": "greeting", "text": "selamat pagi"},
    {"intent": "greeting", "text": "hello"},
    {"intent": "greeting", "text": "good morning"},
    
    # Pricing
    {"intent": "pricing", "text": "berapa harganya"},
    {"intent": "pricing", "text": "harga chattea"},
    {"intent": "pricing", "text": "subscription cost"},
    {"intent": "pricing", "text": "biaya langganan"},
]

responses_dataset = {
    "blast_message": "Untuk blast message, buka menu 'Blast' → Pilih kontak → Tulis pesan → Klik 'Kirim'. Anda bisa mengirim hingga 1000 pesan sekaligus! 📨",
    "schedule_message": "Untuk schedule message, buka 'Schedule' → Pilih kontak → Tulis pesan → Atur tanggal & waktu → Simpan. Pesan akan terkirir otomatis! ⏰",
    "filter_number": "Fitur Filter Number membantu Anda memeriksa nomor mana yang terdaftar di WhatsApp. Buka 'Filter' → Upload daftar nomor → Sistem akan validasi secara otomatis. ✅",
    "panasin_wa": "Panasin WhatsApp adalah fitur untuk mengurangi risiko banned. Sistem membuat beberapa instance chat saling berkomunikasi. Aktifkan di menu 'Settings' → 'Warming Up'. 🔥",
    "help": "Saya bisa bantu Anda dengan:\n1. 📨 Blast Message - Kirim pesan massal\n2. ⏰ Schedule Message - Jadwalkan pesan\n3. ✅ Filter Number - Validasi nomor WA\n4. 🔥 Panasin WA - Hindari banned\n\nSilakan tanyakan fitur yang ingin Anda pelajari!",
    "greeting": "Halo! 👋 Saya Chattea Bot. Saya siap membantu Anda menggunakan aplikasi Chattea. Ada yang bisa saya bantu?",
    "pricing": "Untuk informasi harga dan paket langganan Chattea, silakan hubungi tim sales kami atau kunjungi website resmi. Kami punya berbagai paket sesuai kebutuhan bisnis Anda! 💼",
}

In [6]:
# ============================================================================
# MAIN EXECUTION - NOW WITH PERSISTENCE (only builds once!)
# ============================================================================
import pickle
import os

PERSISTENCE_FILE = "chattea_vectorstore.pkl"

if __name__ == "__main__":
    print("\n" + "=" * 60)
    print("CHATTEA RAG CHATBOT - SEMANTIC SEARCH")
    print("=" * 60)

    # === CHECK IF WE ALREADY HAVE A SAVED VECTOR STORE ===
    if os.path.exists(PERSISTENCE_FILE):
        print(f"Loading saved vector store from {PERSISTENCE_FILE} ...")
        with open(PERSISTENCE_FILE, "rb") as f:
            saved_vector_store = pickle.load(f)
        print("Vector store loaded from disk!")

        # Re-create the chatbot using the saved vector store
        bot = ChatteraRAGChatbot(intents_dataset, responses_dataset, similarity_threshold=0.3)
        bot.vector_store = saved_vector_store  # inject the loaded one
    else:
        print("First time running → building vector store (this takes ~20–30 seconds)...")
        bot = ChatteraRAGChatbot(intents_dataset, responses_dataset, similarity_threshold=0.3)
        
        # === SAVE IT FOR NEXT TIME ===
        print(f"Saving vector store to {PERSISTENCE_FILE} for future runs...")
        with open(PERSISTENCE_FILE, "wb") as f:
            pickle.dump(bot.vector_store, f)
        print("Saved! Next run will be instant.")

    print("=" * 60)

    # === BATCH TEST ===
    test_cases = [
        "gimana cara ngirim pesan ke banyak orang sekaligus?",
        "blst mesage ke customer",
        "atur jadwal kirim pesan otomatis",
        "cek nomor WA yg valid",
        "biar gak kena banned",
        "what features do you have",
        "hallo",
        "bikin pesan terkirim sendiri nanti malam",
    ]
    bot.batch_test(test_cases)



CHATTEA RAG CHATBOT - SEMANTIC SEARCH
First time running → building vector store (this takes ~20–30 seconds)...
Loading embedding model: paraphrase-multilingual-MiniLM-L12-v2
This may take a moment on first run...


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


✓ Model loaded! Embedding dimension: 384

BUILDING KNOWLEDGE BASE
Encoding 53 documents...


Batches: 100%|██████████| 2/2 [00:00<00:00,  5.08it/s]


✓ Added 53 documents to vector store
✓ Total documents: 53

Saving vector store to chattea_vectorstore.pkl for future runs...
Saved! Next run will be instant.

BATCH TESTING

--- Test Case 1 ---
👤 User: gimana cara ngirim pesan ke banyak orang sekaligus?
🤖 Bot: Untuk blast message, buka menu 'Blast' → Pilih kontak → Tulis pesan → Klik 'Kirim'. Anda bisa mengirim hingga 1000 pesan sekaligus! 📨

--- Test Case 2 ---
👤 User: blst mesage ke customer
🤖 Bot: Untuk informasi harga dan paket langganan Chattea, silakan hubungi tim sales kami atau kunjungi website resmi. Kami punya berbagai paket sesuai kebutuhan bisnis Anda! 💼

--- Test Case 3 ---
👤 User: atur jadwal kirim pesan otomatis
🤖 Bot: Untuk schedule message, buka 'Schedule' → Pilih kontak → Tulis pesan → Atur tanggal & waktu → Simpan. Pesan akan terkirir otomatis! ⏰

--- Test Case 4 ---
👤 User: cek nomor WA yg valid
🤖 Bot: Fitur Filter Number membantu Anda memeriksa nomor mana yang terdaftar di WhatsApp. Buka 'Filter' → Upload daftar n

In [ ]:
    # # === INTERACTIVE MODE ===
    # print("\n" + "=" * 60)
    # print("INTERACTIVE MODE (type 'quit' to exit)")
    # print("=" * 60)
    # while True:
    #     user_input = input("\nYou: ")
    #     if user_input.lower() in ['quit', 'exit', 'keluar', 'q']:
    #         print("\nTerima kasih telah menggunakan Chattea Bot!")
    #         break
       
    #     response = bot.chat(user_input, top_k=3, verbose=True)
    #     print(f"Bot: {response}")